# Supplement Ingredient Evidence Collector

**Проєкт:** Автоматизований збір наукових доказів для інгредієнтів харчових добавок
**Version:** v1.0
**Date:** 2025-09-19

---

## Опис
Система для масової обробки 6500+ інгредієнтів з Google Sheets та збору структурованих даних про походження, активні сполуки та добові норми з перевіркою цитат.

## Структура
- **Cell 0:** Config та автентифікація
- **Cell 1:** Google Sheets API
- **Cell 2:** OpenAI API інтеграція
- **Cell 3:** Пошук та верифікація
- **Cell 4:** Batch обробка
- **Cell 5:** Експорт результатів
- **Cell 6:** Quality assurance

In [ ]:
#@title Cell 0 — Configuration and Authentication
#@markdown Налаштування основних параметрів та автентифікація

import os
import json
import datetime
from pathlib import Path

# === CORE CONFIG ===
PROJECT_NAME = "DLSD Evidence Collector"
VERSION = "v1.0"
RUN_ID = f"run_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"

print(f"🚀 {PROJECT_NAME} {VERSION}")
print(f"📊 Run ID: {RUN_ID}")
print(f"⏰ Started: {datetime.datetime.now().isoformat()}")

# === GOOGLE SHEETS CONFIG ===
# Дані з 01_data_sources.md
SHEET_ID_ING = "1kOrSOPgn7IDdA170YJDRBQw4Wt2-Y8uX0PdvCfxY4qA"
SHEET_NAME_ING = "Ingredients_Main"
RANGE_INGREDIENTS = "C2:C"  # основні імена інгредієнтів
RANGE_SYNONYMS_EN_LAT = "E2:E"  # англійські/латинські синоніми

# === OPENAI CONFIG ===
# TODO: Налаштувати через Colab Secrets
OPENAI_MODEL = "gpt-4o"  # або gpt-4o-mini для тестування
MAX_TOKENS = 4096
TEMPERATURE = 0.1  # мінімальна креативність для точності

# === DОЗВОЛЕНІ ДОМЕНИ ===
# З 02_project_constraints.md
ALLOWED_DOMAINS = [
    "nih.gov", "ncbi.nlm.nih.gov", "efsa.europa.eu",
    "examine.com", "consumerlab.com", "sciencedirect.com", "nature.com"
]

# === РОБОЧІ ДИРЕКТОРІЇ ===
WORK_DIR = "/content/dlsd_work"
RESULTS_DIR = f"{WORK_DIR}/results"
LOGS_DIR = f"{WORK_DIR}/logs"

# Створити директорії
os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

print(f"📁 Working directory: {WORK_DIR}")
print(f"💾 Results will be saved to: {RESULTS_DIR}")
print("✅ Configuration completed")

In [ ]:
#@title Cell 1 — Google Sheets API Setup
#@markdown Підключення до Google Sheets та читання інгредієнтів

# Встановлення бібліотек
!pip install -q gspread google-auth google-auth-oauthlib google-auth-httplib2

import gspread
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials
from google.colab import auth, files
import pandas as pd
import re

# === АВТЕНТИФІКАЦІЯ ===
# Варіант 1: Через Google Colab auth (рекомендовано для тестування)
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# Варіант 2: Service Account (для продакшену)
# credentials_path = "/content/service_account.json"  # завантажити файл
# scope = ["https://www.googleapis.com/auth/spreadsheets"]
# creds = Credentials.from_service_account_file(credentials_path, scopes=scope)
# gc = gspread.authorize(creds)

print("🔑 Google Sheets API authenticated")

# === ЧИТАННЯ ДАНИХ ===
def load_ingredients_from_sheets():
    """Завантажує інгредієнти та синоніми з Google Sheets"""
    try:
        # Відкрити таблицю
        sheet = gc.open_by_key(SHEET_ID_ING).worksheet(SHEET_NAME_ING)
        print(f"📋 Opened sheet: {SHEET_NAME_ING}")
        
        # Читання основних імен (C2:C)
        ingredients_raw = sheet.get(RANGE_INGREDIENTS)
        ingredients = [row[0] for row in ingredients_raw if row and row[0].strip()]
        
        # Читання синонімів (E2:E)
        synonyms_raw = sheet.get(RANGE_SYNONYMS_EN_LAT)
        synonyms = [row[0] for row in synonyms_raw if row and row[0].strip()]
        
        print(f"📊 Loaded {len(ingredients)} ingredients")
        print(f"📊 Loaded {len(synonyms)} synonym entries")
        
        return ingredients, synonyms
        
    except Exception as e:
        print(f"❌ Error loading data: {e}")
        return [], []

def build_query_terms(ingredients, synonyms):
    """Формує список query terms з дедуплікацією та фільтрацією"""
    raw_terms = []
    
    # Додати основні імена
    for ingredient in ingredients:
        raw_terms.append(ingredient.strip())
    
    # Додати синоніми (розділені комами)
    for synonym_entry in synonyms:
        for synonym in re.split(r",|;|\n", str(synonym_entry)):
            synonym = synonym.strip()
            if synonym:
                raw_terms.append(synonym)
    
    # Дедуплікація та фільтрація
    query_terms = []
    seen = set()
    
    for term in raw_terms:
        key = term.lower()
        if key not in seen:
            seen.add(key)
            # Пропускаємо ізольовані абревіатури
            if len(term) <= 5 and term.upper() == term:
                continue
            query_terms.append(term)
    
    print(f"🔍 Built {len(query_terms)} unique query terms")
    return query_terms

# === ЗАВАНТАЖЕННЯ ДАНИХ ===
ingredients_list, synonyms_list = load_ingredients_from_sheets()
query_terms = build_query_terms(ingredients_list, synonyms_list)

# Показати перші кілька прикладів
if query_terms:
    print("\n📝 Sample query terms:")
    for i, term in enumerate(query_terms[:10]):
        print(f"  {i+1}. {term}")
    if len(query_terms) > 10:
        print(f"  ... and {len(query_terms) - 10} more")

print("\n✅ Google Sheets data loaded successfully")

In [ ]:
#@title Cell 2 — OpenAI API Integration
#@markdown Налаштування OpenAI API з JSON Schema валідацією

# Встановлення бібліотек
!pip install -q openai jsonschema

import openai
import jsonschema
from google.colab import userdata
import json

# === OPENAI SETUP ===
# Отримати API ключ з Colab Secrets
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    client = openai.OpenAI(api_key=OPENAI_API_KEY)
    print("🔑 OpenAI API key loaded from secrets")
except Exception as e:
    print(f"❌ Please add OPENAI_API_KEY to Colab secrets: {e}")
    # Fallback: manual input
    # OPENAI_API_KEY = getpass.getpass("Enter OpenAI API Key: ")
    # client = openai.OpenAI(api_key=OPENAI_API_KEY)

# === JSON SCHEMA ===
# Схема з 04_system_prompt_json_schema.md
EVIDENCE_SCHEMA = {
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "$id": "https://example.org/schemas/supplement_evidence_v1.json",
    "title": "Supplement Ingredient Evidence",
    "type": "object",
    "required": [
        "ingredient_name_uk",
        "ingredient_name_lat",
        "result_status",
        "sources",
        "citations",
        "provenance"
    ],
    "properties": {
        "ingredient_name_uk": {"type": "string", "minLength": 1},
        "ingredient_name_lat": {"type": "string", "minLength": 1},
        "synonyms": {"type": "array", "items": {"type": "string"}},
        "source_material": {
            "type": "object",
            "properties": {
                "kingdom": {
                    "type": ["string", "null"],
                    "enum": ["Рослини", "Тварини", "Гриби", "Протисти", "Бактерії", "Археї", "Мінерали", "Синтетичне", None]
                },
                "part_or_origin": {"type": ["string", "null"]},
                "description": {"type": ["string", "null"]}
            },
            "required": ["kingdom"],
            "additionalProperties": False
        },
        "active_compounds": {
            "type": "array",
            "items": {
                "type": "object",
                "required": ["name"],
                "properties": {
                    "name": {"type": "string"},
                    "iupac": {"type": ["string", "null"]},
                    "cas": {"type": ["string", "null"]},
                    "concentration": {
                        "type": ["object", "null"],
                        "properties": {
                            "value": {"type": ["number", "null"]},
                            "min_value": {"type": ["number", "null"]},
                            "max_value": {"type": ["number", "null"]},
                            "unit": {
                                "type": ["string", "null"],
                                "enum": ["%", "мг/г", "мкг/г", "мг/мл", "мкг/мл", "ppm", None]
                            }
                        },
                        "additionalProperties": False
                    }
                },
                "additionalProperties": False
            }
        },
        "daily_dose": {
            "type": ["object", "null"],
            "properties": {
                "recommended": {
                    "type": ["object", "null"],
                    "properties": {
                        "value": {"type": ["number", "null"]},
                        "min_value": {"type": ["number", "null"]},
                        "max_value": {"type": ["number", "null"]},
                        "unit": {
                            "type": ["string", "null"],
                            "enum": ["мг/день", "г/день", "мкг/день", "МО/день", "мг/кг/день", None]
                        }
                    },
                    "additionalProperties": False
                },
                "upper_limit": {
                    "type": ["object", "null"],
                    "properties": {
                        "value": {"type": ["number", "null"]},
                        "min_value": {"type": ["number", "null"]},
                        "max_value": {"type": ["number", "null"]},
                        "unit": {
                            "type": ["string", "null"],
                            "enum": ["мг/день", "г/день", "мкг/день", "МО/день", "мг/кг/день", None]
                        }
                    },
                    "additionalProperties": False
                },
                "evidence_type": {
                    "type": ["string", "null"],
                    "enum": ["meta_analysis", "systematic_review", "guideline", "RCT", "observational", "label_claim", "expert_opinion", None]
                }
            },
            "additionalProperties": False
        },
        "result_status": {
            "type": "string",
            "enum": ["found", "partial", "not_found", "ambiguous", "contradictory"]
        },
        "sources": {
            "type": "array",
            "items": {
                "type": "object",
                "required": ["title"],
                "properties": {
                    "title": {"type": "string"},
                    "journal_or_publisher": {"type": ["string", "null"]},
                    "year": {"type": ["integer", "null"]},
                    "url": {"type": ["string", "null"], "format": "uri"},
                    "doi": {"type": ["string", "null"]},
                    "source_priority": {"type": "integer", "minimum": 1, "maximum": 4},
                    "needs_human_review": {"type": "boolean"}
                },
                "additionalProperties": False
            }
        },
        "citations": {
            "type": "array",
            "items": {
                "type": "object",
                "required": ["type", "source_priority"],
                "properties": {
                    "type": {"type": "string", "enum": ["origin", "active_compounds", "daily_dose", "other"]},
                    "title": {"type": ["string", "null"]},
                    "journal_or_publisher": {"type": ["string", "null"]},
                    "year": {"type": ["integer", "null"]},
                    "url": {"type": ["string", "null"], "format": "uri"},
                    "doi": {"type": ["string", "null"]},
                    "exact_quote": {"type": ["string", "null"], "maxLength": 1000},
                    "page_or_section": {"type": ["string", "null"]},
                    "source_priority": {"type": "integer", "minimum": 1, "maximum": 4},
                    "needs_human_review": {"type": "boolean"}
                },
                "additionalProperties": False
            }
        },
        "search_trace": {
            "type": ["array", "null"],
            "items": {
                "type": "object",
                "properties": {
                    "engine": {"type": ["string", "null"]},
                    "query": {"type": ["string", "null"]},
                    "results_checked": {"type": ["integer", "null"]},
                    "notes": {"type": ["string", "null"]}
                },
                "additionalProperties": False
            }
        },
        "provenance": {
            "type": "object",
            "required": ["colab_cell", "model", "retrieved_at"],
            "properties": {
                "colab_cell": {"type": "string"},
                "model": {"type": "string"},
                "model_version": {"type": ["string", "null"]},
                "parser_version": {"type": ["string", "null"]},
                "run_id": {"type": ["string", "null"]},
                "retrieved_at": {"type": "string", "format": "date-time"}
            },
            "additionalProperties": False
        }
    },
    "additionalProperties": False
}

print("📋 JSON Schema loaded")

# === VALIDATION FUNCTION ===
def validate_json_output(data):
    """Валідує JSON за схемою"""
    try:
        jsonschema.validate(instance=data, schema=EVIDENCE_SCHEMA)
        return True, None
    except jsonschema.ValidationError as e:
        return False, str(e)

print("✅ OpenAI API and JSON Schema validation ready")

In [ ]:
#@title Cell 3 — Search and Verification Functions
#@markdown Основні функції пошуку та верифікації даних

import requests
import time
from urllib.parse import urlparse
import hashlib

# === SYSTEM PROMPT ===
# З 04_system_prompt_json_schema.md
SYSTEM_PROMPT = """
Роль: Науковий експерт з фармакогнозії та нутриціології, спеціаліст з БАД і медичних сполук.
Мова відповіді: Українська. Технічні терміни перекладати професійно.

ОБОВ'ЯЗКОВО ДОТРИМУЙСЯ:
1. Нуль галюцинацій. Якщо даних немає, повертай result_status="not_found" і НЕ вигадуй URL/цитати.
2. Точні цитати. Для кожного твердження додавай citations[] з точною цитатою англійською.
3. Пріоритезація джерел: Level 1-4 (найвища до найнижчої довіри).
4. Валідація одиниць. Дози нормалізуй до мг, мкг, г, МО, мг/кг, мг/день, мкг/день.
5. Формат виходу: суто JSON що проходить валідацію.

ПОЛІТИКА ДЖЕРЕЛ:
Level 1: Систематичні огляди, EFSA, FDA, NIH/ODS, WHO
Level 2: RCT у рецензованих журналах
Level 3: Рецензовані журнали 2-3 квартилі
Level 4: Виробники, етикетки (needs_human_review=true)

ДОЗВОЛЕНІ ДОМЕНИ: nih.gov, ncbi.nlm.nih.gov, efsa.europa.eu, examine.com, consumerlab.com, sciencedirect.com, nature.com

СУВОРА JSON СТРУКТУРА:
{
  "ingredient_name_uk": "string (обов'язково)",
  "ingredient_name_lat": "string (обов'язково)", 
  "synonyms": ["string array"],
  "source_material": {
    "kingdom": "one of: Рослини|Тварини|Гриби|Протисти|Бактерії|Археї|Мінерали|Синтетичне",
    "part_or_origin": "string or null",
    "description": "string or null"
  },
  "active_compounds": [{
    "name": "string (обов'язково)",
    "iupac": "string or null",
    "cas": "string or null", 
    "concentration": {
      "value": number,
      "min_value": number,
      "max_value": number,
      "unit": "one of: %|мг/г|мкг/г|мг/мл|мкг/мл|ppm"
    }
  }],
  "daily_dose": {
    "recommended": {
      "value": number,
      "min_value": number, 
      "max_value": number,
      "unit": "one of: мг/день|г/день|мкг/день|МО/день|мг/кг/день"
    },
    "upper_limit": {
      "value": number,
      "min_value": number,
      "max_value": number, 
      "unit": "one of: мг/день|г/день|мкг/день|МО/день|мг/кг/день"
    },
    "evidence_type": "one of: meta_analysis|systematic_review|guideline|RCT|observational|label_claim|expert_opinion|null"
  },
  "result_status": "one of: found|partial|not_found|ambiguous|contradictory",
  "sources": [{
    "title": "string (ОБОВ'ЯЗКОВО)",
    "journal_or_publisher": "string or null",
    "year": integer,
    "url": "string or null",
    "doi": "string or null", 
    "source_priority": integer (1-4),
    "needs_human_review": boolean
  }],
  "citations": [{
    "type": "one of: origin|active_compounds|daily_dose|other",
    "title": "string or null",
    "journal_or_publisher": "string or null",
    "year": integer,
    "url": "string or null",
    "doi": "string or null",
    "exact_quote": "string (до 1000 символів)",
    "page_or_section": "string or null",
    "source_priority": integer (1-4, ОБОВ'ЯЗКОВО)",
    "needs_human_review": boolean
  }],
  "search_trace": [{
    "engine": "string", 
    "query": "string",
    "results_checked": integer,
    "notes": "string"
  }]
}

КРИТИЧНО ВАЖЛИВО:
- sources ЗАВЖДИ повинні мати поле "title" (не "level" чи "description")
- citations ЗАВЖДИ повинні мати поля "type" та "source_priority"
- source_priority тільки числа 1, 2, 3, або 4
- needs_human_review тільки true або false
- evidence_type ТІЛЬКИ: meta_analysis, systematic_review, guideline, RCT, observational, label_claim, expert_opinion, або null
- НЕ використовуй "not_found" для evidence_type - використай null
- Якщо даних немає - порожні масиви [], не null для масивів
- Якщо немає daily_dose даних - встанови daily_dose: null

Повертай ТІЛЬКИ валідний JSON без додаткових коментарів або пояснень.
"""

def create_user_prompt(ingredient_name, synonyms=None):
    """Створює промпт для конкретного інгредієнта"""
    prompt = f"""
Знайди наукові дані про інгредієнт: {ingredient_name}
"""
    
    if synonyms:
        prompt += f"\nСиноніми: {', '.join(synonyms[:5])}"  # обмежити до 5 синонімів
    
    prompt += f"""

Шукай інформацію про:
1. Походження (source_material): царство, частина рослини/тварини
2. Активні сполуки (active_compounds): назви, CAS/IUPAC якщо є, концентрації
3. Добові дози (daily_dose): рекомендовані та верхні межі з одиницями

ОБОВ'ЯЗКОВО:
- Заповни ingredient_name_uk: "{ingredient_name}" (українською)
- Заповни ingredient_name_lat: "наукова латинська назва"
- Для sources використовуй ТІЛЬКИ: title, journal_or_publisher, year, url, doi, source_priority (1-4), needs_human_review
- Для citations використовуй ТІЛЬКИ: type, title, journal_or_publisher, year, url, doi, exact_quote, page_or_section, source_priority (1-4), needs_human_review
- evidence_type ТІЛЬКИ: meta_analysis, systematic_review, guideline, RCT, observational, label_claim, expert_opinion, або null
- Не використовуй поля level, description, "not_found" чи інші неочікувані значення

Повертай лише валідний JSON згідно зі схемою вище.
"""
    
    return prompt

def fetch_evidence_via_openai(ingredient_name, synonyms=None):
    """Отримує дані через OpenAI API"""
    try:
        user_prompt = create_user_prompt(ingredient_name, synonyms)
        
        response = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}
            ],
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            response_format={"type": "json_object"}  # Примусити JSON відповідь
        )
        
        content = response.choices[0].message.content
        
        # Парсинг JSON
        try:
            data = json.loads(content)
        except json.JSONDecodeError as e:
            print(f"❌ JSON parsing error for {ingredient_name}: {e}")
            return None
        
        # Додати provenance
        data["provenance"] = {
            "colab_cell": "Cell 3 — Evidence Fetch",
            "model": OPENAI_MODEL,
            "model_version": response.model,
            "run_id": RUN_ID,
            "retrieved_at": datetime.datetime.now().isoformat()
        }
        
        # Валідація
        is_valid, error = validate_json_output(data)
        if not is_valid:
            print(f"❌ Schema validation failed for {ingredient_name}: {error}")
            print(f"📋 Returned data structure: {list(data.keys()) if isinstance(data, dict) else 'not dict'}")
            if isinstance(data, dict) and 'sources' in data:
                print(f"📋 Sources structure: {[list(s.keys()) if isinstance(s, dict) else s for s in data['sources'][:2]]}")
            return None
        
        return data
        
    except Exception as e:
        print(f"❌ OpenAI API error for {ingredient_name}: {e}")
        return None

def verify_citations(data):
    """Перевіряє доступність URL у цитатах"""
    if not data or "citations" not in data:
        return data
    
    verified_citations = []
    
    for citation in data["citations"]:
        citation["verified"] = False
        
        url = citation.get("url")
        if url:
            try:
                # Перевірка домену
                domain = urlparse(url).netloc.lower()
                domain_allowed = any(allowed in domain for allowed in ALLOWED_DOMAINS)
                
                if domain_allowed:
                    # HTTP перевірка (тільки HEAD запит)
                    response = requests.head(url, timeout=10, allow_redirects=True)
                    if response.status_code == 200:
                        citation["verified"] = True
                    else:
                        print(f"⚠️  URL not accessible: {url} (status: {response.status_code})")
                else:
                    print(f"⚠️  Domain not allowed: {domain}")
                    citation["needs_human_review"] = True
                    
            except Exception as e:
                print(f"⚠️  URL verification failed: {url} - {e}")
        
        verified_citations.append(citation)
    
    data["citations"] = verified_citations
    return data

def fetch_and_verify(ingredient_name, synonyms=None):
    """Головна функція: отримує та верифікує дані"""
    print(f"🔍 Processing: {ingredient_name}")
    
    # Отримати дані через OpenAI
    data = fetch_evidence_via_openai(ingredient_name, synonyms)
    
    if not data:
        return None
    
    # Верифікувати цитати
    data = verify_citations(data)
    
    # Статистика верифікації
    total_citations = len(data.get("citations", []))
    verified_citations = sum(1 for c in data.get("citations", []) if c.get("verified", False))
    
    print(f"✅ Completed {ingredient_name}: {verified_citations}/{total_citations} citations verified")
    
    return data

print("🔍 Search and verification functions ready")
print("📋 System prompt loaded with detailed JSON schema requirements")
print("✅ Cell 3 completed")

In [ ]:
#@title Cell 4 — Batch Processing
#@markdown Масова обробка інгредієнтів з контролем помилок

import time
from tqdm import tqdm
import pickle

# === BATCH SETTINGS ===
BATCH_SIZE = 10  # обробляти по 10 інгредієнтів за раз
DELAY_BETWEEN_REQUESTS = 1  # затримка в секундах між запитами
MAX_RETRIES = 3  # максимум спроб при помилці
CHECKPOINT_FREQUENCY = 50  # зберігати проміжні результати кожні 50 елементів

def process_ingredient_batch(ingredients_batch, start_index=0):
    """Обробляє batch інгредієнтів"""
    results = []
    errors = []
    
    for i, ingredient in enumerate(tqdm(ingredients_batch, desc="Processing batch")):
        current_index = start_index + i
        
        # Пошук синонімів для поточного інгредієнта
        ingredient_synonyms = []
        if current_index < len(synonyms_list):
            synonym_entry = synonyms_list[current_index]
            if synonym_entry:
                ingredient_synonyms = [s.strip() for s in synonym_entry.split(',') if s.strip()]
        
        retry_count = 0
        success = False
        
        while retry_count < MAX_RETRIES and not success:
            try:
                result = fetch_and_verify(ingredient, ingredient_synonyms)
                
                if result:
                    results.append({
                        "index": current_index,
                        "ingredient": ingredient,
                        "data": result,
                        "timestamp": datetime.datetime.now().isoformat()
                    })
                    success = True
                else:
                    retry_count += 1
                    if retry_count < MAX_RETRIES:
                        print(f"⚠️  Retry {retry_count}/{MAX_RETRIES} for {ingredient}")
                        time.sleep(DELAY_BETWEEN_REQUESTS * 2)  # подвійна затримка при ретраї
                
            except Exception as e:
                retry_count += 1
                error_info = {
                    "index": current_index,
                    "ingredient": ingredient,
                    "error": str(e),
                    "retry": retry_count,
                    "timestamp": datetime.datetime.now().isoformat()
                }
                
                if retry_count >= MAX_RETRIES:
                    errors.append(error_info)
                    print(f"❌ Failed after {MAX_RETRIES} retries: {ingredient} - {e}")
                else:
                    print(f"⚠️  Error on retry {retry_count}: {ingredient} - {e}")
                    time.sleep(DELAY_BETWEEN_REQUESTS * 2)
        
        # Затримка між запитами
        if i < len(ingredients_batch) - 1:  # не чекати після останнього елементу
            time.sleep(DELAY_BETWEEN_REQUESTS)
    
    return results, errors

def save_checkpoint(results, errors, checkpoint_index):
    """Зберігає проміжні результати"""
    checkpoint_file = f"{RESULTS_DIR}/checkpoint_{checkpoint_index}.pkl"
    
    checkpoint_data = {
        "results": results,
        "errors": errors,
        "checkpoint_index": checkpoint_index,
        "timestamp": datetime.datetime.now().isoformat(),
        "run_id": RUN_ID
    }
    
    with open(checkpoint_file, 'wb') as f:
        pickle.dump(checkpoint_data, f)
    
    print(f"💾 Checkpoint saved: {checkpoint_file}")

def load_checkpoint(checkpoint_index):
    """Завантажує проміжні результати"""
    checkpoint_file = f"{RESULTS_DIR}/checkpoint_{checkpoint_index}.pkl"
    
    try:
        with open(checkpoint_file, 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:
        return None

def run_full_batch_processing(start_from_checkpoint=None):
    """Запускає повну обробку всіх інгредієнтів"""
    print(f"🚀 Starting batch processing of {len(query_terms)} ingredients")
    print(f"⚙️  Batch size: {BATCH_SIZE}")
    print(f"⏱️  Delay between requests: {DELAY_BETWEEN_REQUESTS}s")
    print(f"🔄 Max retries: {MAX_RETRIES}")
    
    all_results = []
    all_errors = []
    start_index = 0
    
    # Відновлення з checkpoint
    if start_from_checkpoint:
        checkpoint_data = load_checkpoint(start_from_checkpoint)
        if checkpoint_data:
            all_results = checkpoint_data["results"]
            all_errors = checkpoint_data["errors"]
            start_index = checkpoint_data["checkpoint_index"]
            print(f"🔄 Resuming from checkpoint {start_from_checkpoint} (index {start_index})")
    
    # Обробка batches
    total_batches = (len(query_terms) - start_index + BATCH_SIZE - 1) // BATCH_SIZE
    
    for batch_num in range(total_batches):
        batch_start = start_index + batch_num * BATCH_SIZE
        batch_end = min(batch_start + BATCH_SIZE, len(query_terms))
        
        print(f"\n📦 Processing batch {batch_num + 1}/{total_batches} (items {batch_start}-{batch_end-1})")
        
        batch_ingredients = query_terms[batch_start:batch_end]
        batch_results, batch_errors = process_ingredient_batch(batch_ingredients, batch_start)
        
        all_results.extend(batch_results)
        all_errors.extend(batch_errors)
        
        # Checkpoint збереження
        if (batch_num + 1) % (CHECKPOINT_FREQUENCY // BATCH_SIZE) == 0 or batch_end >= len(query_terms):
            save_checkpoint(all_results, all_errors, batch_end)
        
        # Статистика прогресу
        processed = len(all_results)
        failed = len(all_errors)
        total_processed = processed + failed
        
        print(f"📊 Progress: {total_processed}/{len(query_terms)} processed, {processed} successful, {failed} failed")
    
    print(f"\n✅ Batch processing completed!")
    print(f"📊 Final stats: {len(all_results)} successful, {len(all_errors)} failed")
    
    return all_results, all_errors

# === TEST RUN ===
def run_test_batch(num_items=5):
    """Тестовий запуск на невеликій кількості інгредієнтів"""
    print(f"🧪 Running test batch with {num_items} ingredients")
    
    test_ingredients = query_terms[:num_items]
    test_results, test_errors = process_ingredient_batch(test_ingredients)
    
    print(f"\n🧪 Test completed: {len(test_results)} successful, {len(test_errors)} failed")
    
    # Показати приклад результату
    if test_results:
        print("\n📋 Sample result:")
        sample = test_results[0]
        print(f"  Ingredient: {sample['ingredient']}")
        print(f"  Status: {sample['data'].get('result_status', 'unknown')}")
        print(f"  Citations: {len(sample['data'].get('citations', []))}")
    
    return test_results, test_errors

print("⚡ Batch processing functions ready")
print("🧪 Use run_test_batch(5) for testing")
print("🚀 Use run_full_batch_processing() for full processing")
print("✅ Cell 4 completed")

In [ ]:
#@title Cell 5 — Results Export
#@markdown Експорт результатів у Google Sheets, JSONL та CSV

import pandas as pd
import json
import gspread
from google.colab import files

def export_to_jsonl(results, filename=None):
    """Експорт у JSONL формат"""
    if not filename:
        filename = f"{RESULTS_DIR}/dlsd_results_{RUN_ID}.jsonl"
    
    with open(filename, 'w', encoding='utf-8') as f:
        for result in results:
            # Записати тільки дані без метаінформації
            json.dump(result['data'], f, ensure_ascii=False)
            f.write('\n')
    
    print(f"📄 JSONL exported: {filename} ({len(results)} records)")
    return filename

def export_to_csv(results, filename=None):
    """Експорт у CSV формат (спрощена таблиця)"""
    if not filename:
        filename = f"{RESULTS_DIR}/dlsd_results_{RUN_ID}.csv"
    
    # Підготувати дані для CSV
    csv_data = []
    
    for result in results:
        data = result['data']
        
        # Базові поля
        row = {
            'index': result.get('index', ''),
            'ingredient_name_uk': data.get('ingredient_name_uk', ''),
            'ingredient_name_lat': data.get('ingredient_name_lat', ''),
            'result_status': data.get('result_status', ''),
            'kingdom': data.get('source_material', {}).get('kingdom', '') if data.get('source_material') else '',
            'part_or_origin': data.get('source_material', {}).get('part_or_origin', '') if data.get('source_material') else '',
        }
        
        # Активні сполуки (перші 3)
        compounds = data.get('active_compounds', [])
        for i in range(3):
            if i < len(compounds):
                row[f'compound_{i+1}_name'] = compounds[i].get('name', '')
                row[f'compound_{i+1}_cas'] = compounds[i].get('cas', '')
            else:
                row[f'compound_{i+1}_name'] = ''
                row[f'compound_{i+1}_cas'] = ''
        
        # Добові дози
        daily_dose = data.get('daily_dose', {})
        if daily_dose and daily_dose.get('recommended'):
            rec = daily_dose['recommended']
            row['recommended_dose_min'] = rec.get('min_value', '')
            row['recommended_dose_max'] = rec.get('max_value', '')
            row['recommended_dose_value'] = rec.get('value', '')
            row['recommended_dose_unit'] = rec.get('unit', '')
        else:
            row['recommended_dose_min'] = ''
            row['recommended_dose_max'] = ''
            row['recommended_dose_value'] = ''
            row['recommended_dose_unit'] = ''
        
        # Метадані
        row['total_sources'] = len(data.get('sources', []))
        row['total_citations'] = len(data.get('citations', []))
        row['verified_citations'] = len([c for c in data.get('citations', []) if c.get('verified', False)])
        row['needs_human_review'] = any(c.get('needs_human_review', False) for c in data.get('citations', []))
        row['retrieved_at'] = data.get('provenance', {}).get('retrieved_at', '')
        
        csv_data.append(row)
    
    # Створити DataFrame та зберегти
    df = pd.DataFrame(csv_data)
    df.to_csv(filename, index=False, encoding='utf-8')
    
    print(f"📊 CSV exported: {filename} ({len(csv_data)} records, {len(df.columns)} columns)")
    return filename

def create_results_sheet_if_not_exists():
    """Створює аркуш Results_Main якщо не існує"""
    try:
        workbook = gc.open_by_key(SHEET_ID_ING)
        
        # Перевірити чи існує аркуш
        try:
            results_sheet = workbook.worksheet("Results_Main")
            print("📋 Results_Main sheet already exists")
            return results_sheet
        except gspread.WorksheetNotFound:
            # Створити новий аркуш
            results_sheet = workbook.add_worksheet(title="Results_Main", rows=10000, cols=50)
            
            # Додати заголовки
            headers = [
                'Index', 'Ingredient_UK', 'Ingredient_LAT', 'Status', 'Kingdom', 'Part_Origin',
                'Compound_1', 'CAS_1', 'Compound_2', 'CAS_2', 'Compound_3', 'CAS_3',
                'Dose_Min', 'Dose_Max', 'Dose_Value', 'Dose_Unit',
                'Total_Sources', 'Total_Citations', 'Verified_Citations', 'Needs_Review',
                'Retrieved_At', 'Run_ID'
            ]
            
            results_sheet.update('A1', [headers])
            print("📋 Results_Main sheet created with headers")
            return results_sheet
            
    except Exception as e:
        print(f"❌ Error creating Results_Main sheet: {e}")
        return None

def export_to_google_sheets(results):
    """Експорт результатів у Google Sheets"""
    results_sheet = create_results_sheet_if_not_exists()
    
    if not results_sheet:
        print("❌ Cannot export to Google Sheets - sheet creation failed")
        return False
    
    try:
        # Підготувати дані для Google Sheets
        sheet_data = []
        
        for result in results:
            data = result['data']
            
            # Активні сполуки
            compounds = data.get('active_compounds', [])
            compound_names = [c.get('name', '') for c in compounds[:3]]
            compound_cas = [c.get('cas', '') for c in compounds[:3]]
            
            # Додати порожні значення якщо менше 3 сполук
            while len(compound_names) < 3:
                compound_names.append('')
                compound_cas.append('')
            
            # Добові дози
            daily_dose = data.get('daily_dose', {})
            dose_info = ['', '', '', '']
            if daily_dose and daily_dose.get('recommended'):
                rec = daily_dose['recommended']
                dose_info = [
                    rec.get('min_value', ''),
                    rec.get('max_value', ''),
                    rec.get('value', ''),
                    rec.get('unit', '')
                ]
            
            row = [
                result.get('index', ''),
                data.get('ingredient_name_uk', ''),
                data.get('ingredient_name_lat', ''),
                data.get('result_status', ''),
                data.get('source_material', {}).get('kingdom', '') if data.get('source_material') else '',
                data.get('source_material', {}).get('part_or_origin', '') if data.get('source_material') else '',
                *compound_names,
                *compound_cas,
                *dose_info,
                len(data.get('sources', [])),
                len(data.get('citations', [])),
                len([c for c in data.get('citations', []) if c.get('verified', False)]),
                any(c.get('needs_human_review', False) for c in data.get('citations', [])),
                data.get('provenance', {}).get('retrieved_at', ''),
                RUN_ID
            ]
            
            sheet_data.append(row)
        
        # Знайти останній рядок з даними
        existing_data = results_sheet.get_all_values()
        last_row = len(existing_data)
        
        # Додати нові дані
        if sheet_data:
            start_cell = f"A{last_row + 1}"
            results_sheet.update(start_cell, sheet_data)
            
            print(f"📊 Google Sheets updated: {len(sheet_data)} records added starting from row {last_row + 1}")
            print(f"🔗 Sheet URL: https://docs.google.com/spreadsheets/d/{SHEET_ID_ING}/edit#gid={results_sheet.id}")
        
        return True
        
    except Exception as e:
        print(f"❌ Error exporting to Google Sheets: {e}")
        return False

def export_all_formats(results, download_files=True):
    """Експорт у всі формати"""
    if not results:
        print("⚠️  No results to export")
        return
    
    print(f"📤 Starting export of {len(results)} results...")
    
    # JSONL
    jsonl_file = export_to_jsonl(results)
    
    # CSV
    csv_file = export_to_csv(results)
    
    # Google Sheets
    sheets_success = export_to_google_sheets(results)
    
    # Завантажити файли локально
    if download_files:
        try:
            files.download(jsonl_file)
            files.download(csv_file)
            print("💾 Files downloaded to local machine")
        except Exception as e:
            print(f"⚠️  Download failed: {e}")
    
    print(f"\n✅ Export completed:")
    print(f"  📄 JSONL: {jsonl_file}")
    print(f"  📊 CSV: {csv_file}")
    print(f"  📋 Google Sheets: {'✅ Success' if sheets_success else '❌ Failed'}")

print("📤 Export functions ready")
print("💾 Use export_all_formats(results) to export in all formats")
print("✅ Cell 5 completed")

In [ ]:
#@title Cell 6 — Quality Assurance
#@markdown Аналіз якості результатів та генерація звітів

import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import numpy as np

def analyze_results_quality(results):
    """Аналізує якість отриманих результатів"""
    if not results:
        print("⚠️  No results to analyze")
        return {}
    
    analysis = {
        'total_processed': len(results),
        'status_distribution': Counter(),
        'source_priority_distribution': Counter(),
        'verification_stats': {
            'total_citations': 0,
            'verified_citations': 0,
            'needs_human_review': 0
        },
        'kingdom_distribution': Counter(),
        'compounds_stats': {
            'ingredients_with_compounds': 0,
            'total_compounds': 0,
            'avg_compounds_per_ingredient': 0
        },
        'dose_stats': {
            'ingredients_with_doses': 0,
            'evidence_types': Counter()
        }
    }
    
    for result in results:
        data = result['data']
        
        # Статус результатів
        status = data.get('result_status', 'unknown')
        analysis['status_distribution'][status] += 1
        
        # Аналіз джерел
        for source in data.get('sources', []):
            priority = source.get('source_priority', 0)
            analysis['source_priority_distribution'][f'Level {priority}'] += 1
        
        # Аналіз цитат
        citations = data.get('citations', [])
        analysis['verification_stats']['total_citations'] += len(citations)
        
        for citation in citations:
            if citation.get('verified', False):
                analysis['verification_stats']['verified_citations'] += 1
            if citation.get('needs_human_review', False):
                analysis['verification_stats']['needs_human_review'] += 1
        
        # Аналіз source_material
        source_material = data.get('source_material', {})
        if source_material:
            kingdom = source_material.get('kingdom', 'Unknown')
            analysis['kingdom_distribution'][kingdom] += 1
        
        # Аналіз активних сполук
        compounds = data.get('active_compounds', [])
        if compounds:
            analysis['compounds_stats']['ingredients_with_compounds'] += 1
            analysis['compounds_stats']['total_compounds'] += len(compounds)
        
        # Аналіз доз
        daily_dose = data.get('daily_dose', {})
        if daily_dose:
            analysis['dose_stats']['ingredients_with_doses'] += 1
            evidence_type = daily_dose.get('evidence_type', 'unknown')
            analysis['dose_stats']['evidence_types'][evidence_type] += 1
    
    # Обчислити середні значення
    if analysis['compounds_stats']['ingredients_with_compounds'] > 0:
        analysis['compounds_stats']['avg_compounds_per_ingredient'] = (
            analysis['compounds_stats']['total_compounds'] / 
            analysis['compounds_stats']['ingredients_with_compounds']
        )
    
    return analysis

def print_quality_report(analysis):
    """Виводить звіт про якість результатів"""
    print("\n" + "="*60)
    print("📊 QUALITY ANALYSIS REPORT")
    print("="*60)
    
    # Загальна статистика
    print(f"\n📈 GENERAL STATISTICS")
    print(f"  Total processed: {analysis['total_processed']}")
    
    # Розподіл статусів
    print(f"\n📋 RESULT STATUS DISTRIBUTION")
    for status, count in analysis['status_distribution'].most_common():
        percentage = (count / analysis['total_processed']) * 100
        print(f"  {status}: {count} ({percentage:.1f}%)")
    
    # Статистика цитат
    vs = analysis['verification_stats']
    print(f"\n🔍 CITATION VERIFICATION")
    print(f"  Total citations: {vs['total_citations']}")
    print(f"  Verified citations: {vs['verified_citations']}")
    print(f"  Need human review: {vs['needs_human_review']}")
    
    if vs['total_citations'] > 0:
        verification_rate = (vs['verified_citations'] / vs['total_citations']) * 100
        review_rate = (vs['needs_human_review'] / vs['total_citations']) * 100
        print(f"  Verification rate: {verification_rate:.1f}%")
        print(f"  Human review rate: {review_rate:.1f}%")
    
    # Розподіл джерел за пріоритетом
    print(f"\n🏆 SOURCE PRIORITY DISTRIBUTION")
    for priority, count in sorted(analysis['source_priority_distribution'].items()):
        print(f"  {priority}: {count}")
    
    # Розподіл за царствами
    print(f"\n🌱 KINGDOM DISTRIBUTION")
    for kingdom, count in analysis['kingdom_distribution'].most_common():
        percentage = (count / analysis['total_processed']) * 100
        print(f"  {kingdom}: {count} ({percentage:.1f}%)")
    
    # Статистика сполук
    cs = analysis['compounds_stats']
    print(f"\n🧪 ACTIVE COMPOUNDS STATISTICS")
    print(f"  Ingredients with compounds: {cs['ingredients_with_compounds']}")
    print(f"  Total compounds found: {cs['total_compounds']}")
    print(f"  Average compounds per ingredient: {cs['avg_compounds_per_ingredient']:.1f}")
    
    # Статистика доз
    ds = analysis['dose_stats']
    print(f"\n💊 DOSAGE STATISTICS")
    print(f"  Ingredients with dose info: {ds['ingredients_with_doses']}")
    print(f"  Evidence types:")
    for evidence_type, count in ds['evidence_types'].most_common():
        print(f"    {evidence_type}: {count}")

def create_quality_visualizations(analysis, save_plots=True):
    """Створює візуалізації для аналізу якості"""
    plt.style.use('default')
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('DLSD Evidence Collector - Quality Analysis', fontsize=16, fontweight='bold')
    
    # 1. Розподіл статусів результатів
    statuses = list(analysis['status_distribution'].keys())
    status_counts = list(analysis['status_distribution'].values())
    
    axes[0, 0].pie(status_counts, labels=statuses, autopct='%1.1f%%', startangle=90)
    axes[0, 0].set_title('Result Status Distribution')
    
    # 2. Розподіл пріоритетів джерел
    priorities = list(analysis['source_priority_distribution'].keys())
    priority_counts = list(analysis['source_priority_distribution'].values())
    
    if priorities:
        axes[0, 1].bar(priorities, priority_counts, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
        axes[0, 1].set_title('Source Priority Distribution')
        axes[0, 1].set_ylabel('Count')
        plt.setp(axes[0, 1].xaxis.get_majorticklabels(), rotation=45)
    
    # 3. Розподіл за царствами
    kingdoms = list(analysis['kingdom_distribution'].keys())
    kingdom_counts = list(analysis['kingdom_distribution'].values())
    
    if kingdoms:
        axes[1, 0].barh(kingdoms, kingdom_counts)
        axes[1, 0].set_title('Kingdom Distribution')
        axes[1, 0].set_xlabel('Count')
    
    # 4. Статистика верифікації
    vs = analysis['verification_stats']
    verification_data = {
        'Verified': vs['verified_citations'],
        'Not Verified': vs['total_citations'] - vs['verified_citations'],
        'Need Review': vs['needs_human_review']
    }
    
    verification_labels = list(verification_data.keys())
    verification_values = list(verification_data.values())
    
    axes[1, 1].bar(verification_labels, verification_values, 
                   color=['#2ca02c', '#d62728', '#ff7f0e'])
    axes[1, 1].set_title('Citation Verification Status')
    axes[1, 1].set_ylabel('Count')
    plt.setp(axes[1, 1].xaxis.get_majorticklabels(), rotation=45)
    
    plt.tight_layout()
    
    if save_plots:
        plot_filename = f"{RESULTS_DIR}/quality_analysis_{RUN_ID}.png"
        plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
        print(f"📊 Quality analysis plot saved: {plot_filename}")
    
    plt.show()

def generate_quality_report(results, create_visualizations=True):
    """Генерує повний звіт про якість результатів"""
    print("🔍 Analyzing results quality...")
    
    # Провести аналіз
    analysis = analyze_results_quality(results)
    
    # Вивести звіт
    print_quality_report(analysis)
    
    # Створити візуалізації
    if create_visualizations and results:
        create_quality_visualizations(analysis)
    
    # Зберегти аналіз у JSON
    analysis_filename = f"{RESULTS_DIR}/quality_analysis_{RUN_ID}.json"
    with open(analysis_filename, 'w', encoding='utf-8') as f:
        json.dump(analysis, f, ensure_ascii=False, indent=2, default=str)
    
    print(f"\n💾 Quality analysis saved: {analysis_filename}")
    
    return analysis

def sample_manual_verification(results, sample_size=10):
    """Вибирає випадкову вибірку для ручної перевірки"""
    if not results:
        print("⚠️  No results for manual verification")
        return []
    
    # Випадкова вибірка
    import random
    sample_size = min(sample_size, len(results))
    sample = random.sample(results, sample_size)
    
    print(f"\n🔍 MANUAL VERIFICATION SAMPLE ({sample_size} items)")
    print("="*60)
    
    for i, result in enumerate(sample, 1):
        data = result['data']
        print(f"\n{i}. {data.get('ingredient_name_uk', 'Unknown')} ({data.get('ingredient_name_lat', 'Unknown')})")
        print(f"   Status: {data.get('result_status', 'unknown')}")
        print(f"   Citations: {len(data.get('citations', []))}")
        
        # Показати першу цитату
        citations = data.get('citations', [])
        if citations:
            first_citation = citations[0]
            print(f"   Sample citation: {first_citation.get('title', 'No title')[:50]}...")
            print(f"   URL: {first_citation.get('url', 'No URL')}")
            print(f"   Verified: {first_citation.get('verified', False)}")
    
    return sample

print("📊 Quality assurance functions ready")
print("🔍 Use generate_quality_report(results) for full analysis")
print("🎯 Use sample_manual_verification(results, 10) for manual checking")
print("✅ Cell 6 completed")

print("\n" + "="*60)
print("🎉 DLSD EVIDENCE COLLECTOR NOTEBOOK READY!")
print("="*60)
print("\n📋 Usage:")
print("1. Run cells 0-2 to set up APIs and load data")
print("2. Use run_test_batch(5) to test with 5 ingredients")
print("3. Use run_full_batch_processing() for full processing")
print("4. Use export_all_formats(results) to save results")
print("5. Use generate_quality_report(results) for analysis")
print("\n🚀 Ready to process 6500+ ingredients!")